In [1]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('../data/songs.csv', index_col=0)

In [ ]:
data.Title = data['Title'].str.lower()
data.Artist = data['Artist'].str.lower()

In [ ]:
data.head()

# Get Spotify URI

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm_notebook
import spotipy
import spotipy.util as util
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

In [3]:
token = util.oauth2.SpotifyClientCredentials(client_id='73d7e59bc32d4de39730cf12f4fc98ee', client_secret='a8025ef05dd1463995e3c670806ea6cb')
cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)
sp = spotipy.Spotify(auth=cache_token)

In [ ]:
titles = list(data.Title)
artists = list(data.Artist)

In [ ]:
spotify_uri = list()
errors = list()

In [ ]:
def get_spotify_uri(title, artist):
    title_clean = re.sub(r"[,.;@#?!&$%()]+", ' ', title)
    title_clean = re.sub('\s+', ' ', title_clean).strip()
    artist_clean = re.sub('\s+', ' ', artist).strip()
    
    query = title_clean + " " + artist_clean
    
    search = sp.search(q=query, limit=50, offset=0, type='track', market='US')
    search_items = search['tracks']['items']
    
    for i in range(len(search_items)):
        spotify_title = search_items[i]['name']
        spotify_artist = search_items[i]['artists'][0]['name']
        
        spotify_title_clean = re.sub(r"[,.;@#?!&$%()]+", ' ', spotify_title)
        spotify_title_clean = re.sub('\s+', ' ', title_clean).strip().lower()
        spotify_artist_clean = spotify_artist.lower().strip().lower()
        
        fuzzy_title_match = fuzz.token_set_ratio(title_clean, spotify_title_clean)
        fuzzy_artist_match = fuzz.token_set_ratio(artist_clean, spotify_artist_clean)
        fuzzy_match = (fuzzy_title_match + fuzzy_artist_match) / 2

        if (fuzzy_title_match >= 90) and (fuzzy_artist_match >= 50) and fuzzy_match >= 75:
            uri = search_items[i]['id']
            return uri
    return 0

In [ ]:
temp = list()

for i in tqdm_notebook(range(0, 1000)):
    uri = get_spotify_uri(titles[i], artists[i])
    
    if uri != 0:
        temp.append(uri)
    else:
        temp.append(uri)
        errors.append(i)
        
spotify_uri = spotify_uri + temp

In [ ]:
data['URI'] = spotify_uri
data = data[data.URI != 0]

In [ ]:
# data.to_csv('songs_w_uri.csv')

# Get Spotify Features

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm_notebook
import spotipy
import spotipy.util as util
import re

In [ ]:
token = util.oauth2.SpotifyClientCredentials(client_id='73d7e59bc32d4de39730cf12f4fc98ee', client_secret='a8025ef05dd1463995e3c670806ea6cb')
cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)
sp = spotipy.Spotify(auth=cache_token)

In [ ]:
data = pd.read_csv('../data/songs_w_uri.csv', index_col=0)

In [ ]:
uris = list(data.URI)

danceability_list = list()
energy_list = list()
key_list = list()
loudness_list = list()
mode_list = list()
speechiness_list = list()
acousticness_list = list()
instrumentalness_list = list()
liveness_list = list()
valence_list = list()
tempo_list = list()
duration_list = list()
time_signature_list= list()

In [ ]:
def get_audio_features(uri):
    search = sp.audio_features(uri)
    if search[0] == None:
        danceability_list.append(np.nan)
        energy_list.append(np.nan)
        key_list.append(np.nan)
        loudness_list.append(np.nan)
        mode_list.append(np.nan)
        speechiness_list.append(np.nan)
        acousticness_list.append(np.nan)
        instrumentalness_list.append(np.nan)
        liveness_list.append(np.nan)
        valence_list.append(np.nan)
        tempo_list.append(np.nan)
        duration_list.append(np.nan)
        time_signature_list.append(np.nan) 
        return ('Error on: ' + str(uri))
    
    search_list = search[0]
    
    danceability_list.append(search_list['danceability'])
    energy_list.append(search_list['energy'])
    key_list.append(search_list['key'])
    loudness_list.append(search_list['loudness'])
    mode_list.append(search_list['mode'])
    speechiness_list.append(search_list['speechiness'])
    acousticness_list.append(search_list['acousticness'])
    instrumentalness_list.append(search_list['instrumentalness'])
    liveness_list.append(search_list['liveness'])
    valence_list.append(search_list['valence'])
    tempo_list.append(search_list['tempo'])
    duration_list.append(search_list['duration_ms'])
    time_signature_list.append(search_list['time_signature'])

In [ ]:
for i in tqdm_notebook(range(len(uris))):
    get_audio_features(uris[i])

In [ ]:
data['Danceability'] = danceability_list
data['Energy'] = energy_list
data['Key'] = key_list
data['Loudness'] = loudness_list
data['Mode'] = mode_list
data['Speechiness'] = speechiness_list
data['Acousticness'] = acousticness_list
data['Instrumentalness'] = instrumentalness_list
data['Liveness'] = liveness_list
data['Valence'] = valence_list
data['Tempo'] = tempo_list
data['Duration'] = duration_list
data['Time_Signature'] = time_signature_list

In [ ]:
data = data.dropna()

In [ ]:
# data.to_csv('songs_w_spotifyapi.csv')

# Get Lyrics from Musixmatch

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from musixmatch import Musixmatch
from tqdm import tqdm_notebook

In [ ]:
api_key = '7d8043070d8b187e54fbcb656e8537aa'
musixmatch = Musixmatch(api_key)

In [ ]:
data = pd.read_csv("../data/songs_w_spotifyapi.csv", index_col=0)

In [ ]:
data['lyrics'] = np.nan
data['explicit'] = np.nan

In [ ]:
lyrics_list = list()
explicit_list = list()
errors = list()

In [ ]:
songs = list(data.Title)
artists = list(data.Artist)

In [ ]:
def get_lyrics(track, artist):
    output = musixmatch.matcher_lyrics_get(track, artist)
    
    if (output['message']['body'] == ''):
        return None
    if (output['message']['body'] == []):
        return None
    else:
        lyrics = output['message']['body']['lyrics']['lyrics_body'].split('***')[0]
        explicit = output['message']['body']['lyrics']['explicit']
        return [lyrics, explicit]

In [ ]:
for i in tqdm_notebook(range(len(songs))):
    song = songs[i]
    artist = artists[i]
    
    output = get_lyrics(song, artist)
    
    if output == None:
        lyrics_list.append(np.nan)
        explicit_list.append(np.nan)
        errors.append(i)
    else:
        lyrics_list.append(output[0])
        explicit_list.append(output[1])

In [ ]:
data.lyrics = lyrics_list
data.explicit = explicit_list

In [ ]:
data = data.dropna()

In [ ]:
# data.to_csv('songs_w_features_lyrics.csv')

# Get Year

In [ ]:
data = pd.read_csv("../data/songs_w_features_lyrics.csv", index_col=0)

In [ ]:
songs = list(data.Title)
artists = list(data.Artist)
years = list()
errors = list()

In [ ]:
def get_song_year(title, artist):
    title_clean = re.sub(r"[,.;@#?!&$%()]+", ' ', title)
    title_clean = re.sub('\s+', ' ', title_clean).strip()
    artist_clean = re.sub('\s+', ' ', artist).strip()
    
    query = title_clean + " " + artist_clean
    
    try:
        search = sp.search(q=query, limit=50, offset=0, type='track')
        search_items = search['tracks']['items']
        year = search_items[0]['album']['release_date']
        return year
    except Exception:
        year = 0
        return year

In [ ]:
for i in tqdm_notebook(range(len(songs))):
    year = get_song_year(songs[i], artists[i])
    
    if year != 0 :
        years.append(year)
    else:
        years.append(year)
        print("Errored on " + str(i))
        errors.append(i)    

In [ ]:
new_years = list()

def clean_year(date):
    y = date.split('-')[0]
    return int(y)

In [ ]:
for i in years:
    if i == 0:
        new_years.append(i)
    else:
        x = clean_year(i)
        new_years.append(x)

In [ ]:
data['Release_Year'] = new_years
data = data[data.Release_Year != 0]

In [ ]:
# data.to_csv('songs_w_features_lyrics_year.csv')

# Get Genre

In [35]:
data = pd.read_csv('../data/songs_w_features_lyrics_year.csv', index_col=0)

In [28]:
artists = list(data.Artist)

In [29]:
def get_first_artist(artist):
    
    #Handling , or and
    for i in range(len(artist)):
        if artist[i] == ',' or artist[i] == '&':
              return artist[0:i]
        if artist[i:i+3] == 'and':
              return artist[0:i]
    return artist

#Helper function to take subgenres of each artist and find the most frequent common genre within potentially 1000 subgenres
def get_common_genres(test_subgenres, common_genre_keywords):
    
    final_genres = []
    genre_frequency_map = {}
  
    #Checking the subgenres to see if they match common genre keywords

    for keyword in common_genre_keywords:
        for subg in test_subgenres:
            if keyword in subg:
                final_genres.append(keyword)

    #If no final genres can be identified, return None.
    if len(final_genres) == 0:
        return "None"

    #Counting the number of each genre keyword and returning the one with the highest count (as the "common genre")

    for genre in final_genres:
        if genre in genre_frequency_map.keys():
            current_value = genre_frequency_map[genre]
            genre_frequency_map[genre] = current_value + 1
        else:
            genre_frequency_map[genre] = 1

    #Getting most frequent common genre.

    max_value = max(genre_frequency_map.values())  # maximum value
    max_keys = [k for k, v in genre_frequency_map.items() if v == max_value]
    return max_keys[0]

In [30]:
def get_genres_for_each_artist(artists):
    
    final_genre_list = []
    #Most popular American Music Genres
    common_genre_keywords = ["rap", "pop", "rock", "country", "alternative", "r&b" "latin", "edm", "seasonal", "jazz", "classical", "metal", "reggae"]
    
    for i in tqdm_notebook(range(len(artists))):
        artist_name = artists[i]
        #Clean Artist Name 
        artist_name = get_first_artist(artist_name)
        #Include stripping code of and etc. here
 
        #stripping regex symbols and getting the prominent artist name
      
        #artist_name = re.sub(r"[,.;@#?!&$-/]+", ' ', artist_name)
        #artist_name = re.sub('\s+', ' ', artist_name).strip()
        #artist_name = artist_name(' ')[0]
 
        #Using the spotify search functionality and extracting the list of subgenres.
        try:
            search = sp.search(q=artist_name, limit=1, offset=0, type='artist', market='US')
        except Exception:
            print('Spotify error for ' + str(i))
            final_genre_list.append('None')
            continue
            

        try:
            list_of_subgenres = search['artists']['items'][0]['genres']
            common_genre = get_common_genres(list_of_subgenres, common_genre_keywords)
            final_genre_list.append(common_genre)
        except IndexError:
            print('No Genre for ' + str(i))
            final_genre_list.append('None')
            continue
    
    return final_genre_list

In [31]:
genres_list = get_genres_for_each_artist(artists)

No Genre for 9
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
No Genre for 230
retrying ...1secs
retrying ...1secs
No Genre for 318
No Genre for 331
retrying ...2secs
retrying ...1secs
No Genre for 482
retrying ...2secs
retrying ...1secs
No Genre for 699
No Genre for 832
retrying ...2secs
No Genre for 1124
retrying ...2secs
retrying ...1secs
No Genre for 1158
retrying ...1secs
retrying ...1secs
retrying ...1secs
No Genre for 1499
No Genre for 1564
retrying ...1secs
retrying ...1secs
No Genre for 1712
No Genre for 1723
No Genre for 1837
No Genre for 1841
retrying ...2secs
retrying ...1secs
No Genre for 1852
No Genre for 1860
No Genre for 1887
No Genre for 1956
retrying ...1secs
No Genre for 1998
Spotify error for 2112
retrying ...1secs
retrying ...1secs
No Genre for 2235
No Genre for 2252
Spotify error for 2273
Spotify error for 2458
Spotify error for 2510
No Genre for 2512
retrying ...2secs
retrying ...1secs
No Genre for 2578
No Genre for 2605
No Genre for 2612

In [36]:
data['Genre'] = genres_list
data = data[data['Genre'] != 'None']

In [38]:
# data.to_csv('songs_complete_data.csv')